# Applying AI in Game -Super Hexagon-

Hyeon Jin Yoo (from Department of biological science in Seoul National University)

이 문서는 코딩기술서에 설명된 실제 구현 코드를 상세하게 서술한 문서이며, 각 셀의 코드의 바로 앞에 간단한 설명 및 필요 사항은 서술해두었다. 목차가 본 코딩기술서와 동일하니 자세한 설명은 코딩기술서의 해당 목록을 참조하면 된다.

## 2. Supervised Learning Process and Result

### 2-1. Measuring Method

#### 1) Capture Image and Get Array

이 과정에서는 pyautogui, numpy 라이브러리를 import하여야 한다. (미설치시 설치가 필요하다.)

In [ ]:
import pyautogui
import numpy as np

게임 창을 특정 위치에 고정시키고, 잘 캡처가 되는지 확인해보자. region 내 숫자를 조정하여 캡처가 되는 범위를 변경할 수 있다.

In [ ]:
pic = pyautogui.screenshot(region=(20, 40, 950, 600)) # 숫자를 조정하여 캡쳐 범위를 확인하자.
pic

이미지를 캡처해서 numpy array로 변환하는 함수이다. 반환된 array는 x축, y축의 픽셀 수와 RGB를 나타내는 3개 값에 대한 차원으로 이루어져있다.

In [ ]:
def get_pic_array():
    pic = pyautogui.screenshot(region=(20, 40, 950, 600)) # 위에서 찾았던 숫자를 대입한다.
    pic_array = np.array(pic)
    return pic_array

#### 2) Find Location of Ship

이 과정은 중심의 좌표를 찾는 파트, 중심으로 부터 떨어진 거리와 회전 각도에 대응하는 좌표를 cos, sin 값을 이용하여 찾아 그 좌표의 픽셀 값을 반환하는 함수를 정의하는 파트, 그리고 커서가 실제로 어디에 있는지를 코딩 기술서의 1, 2, 3번 조건을 가지고 찾는 함수를 정의하는 파트로 구성되어 있다. 앞 두 파트는 커서를 실제로 찾는 함수를 위한 것이며, 중심 좌표를 찾았으면 calculate_color 함수에서 해당 숫자만 바꾸어주면 된다. 중심 좌표를 찾기 위해서 이미지에 원을 그려 원이 그려지는 위치를 확인해볼 것인데, 이 과정에서는 matplotlib과 opencv 라이브러리가 필요하다.

In [ ]:
### 중심 좌표를 정확하게 찾는 방법 ###

import matplotlib.pyplot as plt
import cv2
%matplotlib inline

img = get_pic_array()
cv2.circle(img, (469,300), 70, (0,0,0), 5) # 70은 원의 반지름, 5는 굵기이다.
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) # 원이 가운데 육각형을 기준으로 중심에 위치한다면 중심 좌표라고 볼 수 있다.

In [ ]:
### 중심으로 부터 떨어진 거리와 회전 각도에 대응하는 좌표의 픽셀 값을 찾는 함수 ###

def calculate_color(dist, rad, array):
    return array[int(300-dist*np.cos(rad*np.pi/180)),int(469+dist*np.sin(rad*np.pi/180))][0]

# 300은 y축 중심좌표, 469는 x축 중심좌표로 측정되었는데, 이는 조절될 필요가 있을 수도 있다.
# 마지막에 0을 사용한 것을 RGB 중 R에 대응되는 값을 측정한 것인데, 6스테이지의 경우 G, B 모두 사용이 가능하다.

In [ ]:
### 커서가 몇˚ 만큼 회전한 위치에 있는지를 찾아내는 함수 ###

def measure_radian(pic_array):
    rad = 0
    for j in range(180): # 2˚단위로 Brute Force 탐색
        cnt = 0 # 흰색 픽셀의 굵기를 감지하려는 변수
        seq = 0 # 두 번째로 흰색 감지되는 경우를 찾기 위한 변수(1번 조건)
        if rad != 0: # 커서 위치 찾으면 탐색 종료
            break
        for i in range(35,100,2): # 중심 기준 35픽셀~100픽셀 떨어진 범위 2픽셀씩 전진탐색
            if calculate_color(i, 2*j, pic_array) >= 235: # 흰색 감지
                if seq == 0:
                    seq = 1
                if seq == 2:
                    cnt += 1
            else: # 흰색 미감지
                if seq == 1: # 흰색이 앞에서 감지된 경우 업데이트
                    seq = 2
                if seq == 2 and cnt in [2, 3, 4]: # 굵기가 특정 범위 내에 있어야 커서로 인식(2번 조건)
                    ch = 0
                    for k in range(-4, 5): # 3번 조건 탐색
                        if calculate_color(i+k, 2*j-8, pic_array) >= 235 or \
                        calculate_color(i+k, 2*j+8, pic_array) >= 235:
                            ch = 1
                            break
                    if ch == 0: # 모든 조건 만족시 rad 업데이트
                        rad = 2 * j
                    break
    return rad

#### 3) Measure Distance

앞에서 정의한 커서 위치를 찾는 함수와 원하는 위치의 픽셀을 찾는 함수를 이용하여 픽셀 탐색을 통해 벽이 각 방향에서 어느 정도의 거리에 떨어져 있는지를 측정하는 함수를 정의하였다. 이 함수에서 결과는 length 12의 list형태로 반환된다.

In [ ]:
def measure_distance(pic_array):
    dist_1 = [30] * 6 # 최대값 미리 부여하고 시작(오류 발생 가능성 제거)
    dist_2 = [30] * 6 # 두 번째 벽도 인식
    ch = 0
    rad = measure_radian(pic_array) # 커서 위치 측정
    for i in range(6): # 60˚씩 회전하여 측정
        rads = (rad + 60 * i) % 360
        for dist in range(92, 392, 8): # 중심에서 92픽셀 떨어진 위치 부터 해당 방향으로 8 픽셀씩 전진
            if ch > 0: # 흰색 나온 직후 픽셀 감지 생략
                ch -= 1
                continue
            try:
                if calculate_color(dist, rads , pic_array) < 50: # 메뉴바 부분 픽셀 감지
                    break
                if calculate_color(dist, rads , pic_array) >= 235: # 흰색 픽셀 감지
                    if dist_1[i] == 30: # 아직 벽이 인식이 안되었던 경우
                        dist_1[i] = round((dist - 92) / 10)
                        ch = 5 # 40픽셀 점프하여 다시 계산 유도
                    else:
                        dist_2[i] = round((dist - 92) / 10)
                        break
            except:
                break
    return dist_1 + dist_2 # 0~5번 인덱스는 첫번째 벽(현재위치부터 60도씩 회전), 6~11번 인덱스는 두번째 벽과의 거리이다.

#### 4) Collect Data

게임 오버 상태를 감지하는 함수, input data를 생성하는 함수, 그리고 키보드 상태를 감지하여 label data를 생성하는 함수를 작성하였다. 키보드 상태 감지 및 원격 조작에는 keyboard 라이브러리가 필요하다.

In [ ]:
### 게임 오버 감지 함수 ###

def Is_GameOver(pic_array): # 게임 오버시 검정색으로 바뀌는 부분 인지
    if calculate_color(50, 300 , pic_array) < 2:
        return True
    else:
        return False

In [ ]:
### Input Data 만드는 함수 ###

def make_data():
    pic_array = get_pic_array()
    try:
        if not Is_GameOver(pic_array):
            dist = measure_distance(pic_array)
            return dist
        else:
            return [-1]*12
    except:
        return [-1]*12

In [ ]:
### 키보드 상태 감지하여 Label Data 만드는 함수 ###

import keyboard

def check_direction():
    if keyboard.is_pressed('left'):
        label = 'left'
    elif keyboard.is_pressed('right'):
        label = 'right'
    else:
        label = 'stay'
    return label

In [ ]:
total_data = [] # 수집한 데이터를 모을 리스트
total_label = [] # 수집한 라벨을 모을 리스트

이 코드를 실행시키고 게임을 플레이하면 자동으로 데이터가 모아진다. 게임 오버 상태와 게임 오버 직전의 bad data는 모아지지 않으며, 만약 플레이가 맘에 안든 경우는 n을 누르면 전체 dataset에 저장을 포기하게 된다. 저장을 원하면서 종료하고 싶다면 y를 누르면 된다.

In [ ]:
datas = []
labels = []
ch = 1
while True:
    state = make_data()
    action = check_direction()
    if state[0] != -1: # 게임 오버 상태가 아닌 경우만 데이터 저장
        ch = 1
        datas.append(state)
        labels.append(action)
    else:
        ch -= 1
        if ch == 0: # 게임 오버 화면 지속시 중복 삭제 방지
            last = min(10, len(datas))
            datas = datas[:-last] # bad data 삭제
            labels = labels[:-last]
    if keyboard.is_pressed('y'): # y 누르면 전체 데이터셋에 이번에 모은 데이터 저장
        for data in datas:
            total_data.append(data)
        for label in labels:
            total_label.append(label)
        break
    if keyboard.is_pressed('n'): # n 누르면 이번에 모은 데이터 포기
        break

만약 모은 데이터를 다음 번에 커널 재실행시에도 다시 사용하고 싶다면 pickle 라이브러리를 통해 저장 및 불러오기를 할 수 있다.

In [ ]:
# 데이터 pickle 파일로 저장

import pickle as pkl

with open('data.pickle', 'wb') as f:
    pkl.dump(total_data, f)
    
with open('label.pickle', 'wb') as f:
    pkl.dump(total_label, f)

In [ ]:
# pickle 데이터 리스트로 불러오기

with open('data.pickle', 'rb') as f:
    total_data = pkl.load(f)
    
with open('label.pickle', 'rb') as f:
    total_label = pkl.load(f)

#### 5) Training

라벨은 left, stay, right로 저장이 되어있는데, 학습의 용이함을 위해 0, 1, 2의 정수형 자료로 각각 변환시켰다. train, test dataset 분할을 위해서 sklearn의 train_test_split 함수를 사용하였으며, tensorflow 2 버전을 이용하여 모델을 작성하고 학습시키는 코드를 작성하였다.

In [ ]:
# 라벨 값 정수 변환

label_num = [x.replace('left', '0').replace('stay', '1').replace('right', '2') for x in total_label]
label_num = list(map(int, label_num))
label_num

In [ ]:
# train, test dataset 분할

from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(total_data, label_num, test_size=0.2)

In [ ]:
# Tensorflow 모델 작성

import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(12, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='sigmoid'),
  tf.keras.layers.Dense(128, activation='tanh'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Tensorflow 모델 학습

model.fit(train_data, train_label, epochs=10)
model.evaluate(test_data, test_label, verbose=2)

# Tensorflow 모델 저장

model.save('Measure_Supervised_model.h5')

#### 6) Play Game

모델을 학습하였으니, 이 모델의 예측 결과로 게임을 실행해볼 수 있다. 예측 결과에 따라 키보드를 조작하는 함수도 같이 작성하였다. 게임을 켜고 다음 코드를 실행해보면 된다. 종료를 원하면 n을 누르면 된다.

In [ ]:
# Tensorflow 모델 불러오기

model = tf.keras.models.load_model('Measure_Supervised_model.h5')

In [ ]:
### 예측 결과에 따라 키보드를 조작하는 함수 ###

def move(predict):
    if predict == 0:
        keyboard.release('right arrow')
        keyboard.press('left arrow')
    elif predict == 1:
        keyboard.release('left arrow')
        keyboard.release('right arrow')
    else:
        keyboard.release('left arrow')
        keyboard.press('right arrow')

In [ ]:
# 게임 실행(n 입력하면 종료)

while True:
    while Is_GameOver(get_pic_array()): # 게임 오버 시 자동 재시작
        keyboard.press('space')
        if keyboard.is_pressed('n'):
            break
    keyboard.release('space')
    
    if keyboard.is_pressed('n'):
        break
    
    dist = make_data()
    predict = np.argmax(model.predict([dist])[0])
    move(predict)

#### 7) 번외 : 다른 단계에 적용하기

3단계에 해당 코드를 적용하는 방법을 제시하였다. 위에서 작성했던 함수 중 다음 함수들만 아래와 같이 변경하면 3단계에서 같은 방법을 시도해볼 수 있다.

In [ ]:
def measure_radian(pic_array):
    rad = 0
    for j in range(180):
        cnt = 0
        seq = 0
        if rad != 0:
            break
        for i in range(35,100,2):
            if calculate_color(i, 2*j, pic_array) >= 200: # 역치가 다름
                if seq == 0:
                    seq = 1
                if seq == 2:
                    cnt += 1
            else:
                if seq == 1:
                    seq = 2
                if seq == 2 and cnt in [2, 3, 4]:
                    ch = 0
                    for k in range(-4, 5):
                        if calculate_color(i+k, 2*j-8, pic_array) >= 200 or \
                        calculate_color(i+k, 2*j+8, pic_array) >= 200:
                            ch = 1
                            break
                    if ch == 0:
                        rad = 2 * j
                    break
    return rad

def calculate_color(dist, rad, array): # R, G, B 모두 탐색하여 최대값만 골라냄, 중심 픽셀 값 다를 경우 수정 필요
    return max(array[int(300-dist*np.cos(rad*np.pi/180)),int(469+dist*np.sin(rad*np.pi/180))][0],
               array[int(300-dist*np.cos(rad*np.pi/180)),int(469+dist*np.sin(rad*np.pi/180))][1],
               array[int(300-dist*np.cos(rad*np.pi/180)),int(469+dist*np.sin(rad*np.pi/180))][2])

def measure_distance(pic_array):
    dist_1 = [30] * 6
    dist_2 = [30] * 6
    ch = 0
    rad = measure_radian(pic_array)
    for i in range(6):
        rads = (rad + 60 * i) % 360
        for j in range(92, 392, 8):
            if ch > 0:
                ch -= 1
                continue
            try:
                if calculate_color(j, rads , pic_array) >= 200:
                    if dist_1[i] == 30:
                        dist_1[i] = round((j - 92) / 10)
                        ch = 5
                    else:
                        dist_2[i] = round((j - 92) / 10)
                        break
            except:
                break
    return dist_1 + dist_2

### 2-2. CNN Method

#### 2) Collect Data

2-1에서 사용했던 방법과는 달리 CNN을 위한 input data를 만드려면 이미지 전체의 정보를 이용해야 하므로 다른 형태의 data를 추출해내는 함수가 필요했다. 여기서는 흰색 부분만 흰색으로남기고 나머지는 모두 검정색으로 처리하는 함수를 작성하였으며, 적용 단계는 6단계를 기준으로 하며, 950 * 600의 image size를 기준으로 작성하였다. 아래에서 서술할 모델에 적합한 차원을 가진 numpy array자료형으로 값을 반환할 예정이다. numpy, pyautogui와 opencv 라이브러리가 필요하다.

In [ ]:
def make_data():
    pic_array = get_pic_array()
    cv2.rectangle(pic_array, (0,0),(949,50),(0,0,0), -1) # 시간이 나오는 윗 부분은 노이즈로 작용할 수 있어 제거
    pic_array = cv2.cvtColor(pic_array, cv2.COLOR_RGB2GRAY)
    pic_array = cv2.threshold(pic_array, 235, 255, cv2.THRESH_BINARY)
    return np.expand_dims(pic_array[1], axis = 2)

변경된 make_data 함수를 제외하고 나머지는 모두 2-1의 4)항목에 나와있는 나머지 함수와 코드를 동일하게 사용하면 된다. 단, 이번에는 앞의 예제와는 달리 pickle 파일의 용량이 매우 클 예정이므로, 만약 CPU 내에서 작동 중이었다면, save/load시 주의가 필요하다.

In [ ]:
total_data = [] # 수집한 데이터를 모을 리스트
total_label = [] # 수집한 라벨을 모을 리스트

In [ ]:
datas = []
labels = []
ch = 1
while True:
    state = make_data()
    action = check_direction()
    if not Is_GameOver(get_pic_array()): # 게임 오버 상태가 아닌 경우만 데이터 저장
        ch = 1
        datas.append(state)
        labels.append(action)
    else:
        ch -= 1
        if ch == 0: # 게임 오버 화면 지속시 중복 삭제 방지
            last = min(5, len(datas))
            datas = datas[:-last] # bad data 삭제
            labels = labels[:-last]
    if keyboard.is_pressed('y'): # y 누르면 전체 데이터셋에 이번에 모은 데이터 저장
        for data in datas:
            total_data.append(data)
        for label in labels:
            total_label.append(label)
        break
    if keyboard.is_pressed('n'): # n 누르면 이번에 모은 데이터 포기
        break

#### 3) Training

라벨값 정수변환, train, test dataset 분할 과정은 2-1의 5)항목과 동일하게 진행되지만, CNN을 구현한 모델을 사용할 예정이라 다음과 같이 모델을 변경하였다. 마찬가지로 Tensorflow 2 버전 프레임워크를 사용하였는데, 단, 이번에는 input data dimension이 매우 크므로, GPU 내에서 학습을 진행하는 것을 권장한다. 또한, 모델의 형태가 달라 input으로 list가 아닌 numpy array를 이용한다는 점도 다른 점이다.

In [ ]:
label_num = [x.replace('left', '0').replace('stay', '1').replace('right', '2') for x in total_label]
label_num = list(map(int, label_num))

train_data, test_data, train_label, test_label = train_test_split(total_data, label_num, test_size=0.2)

train_data = np.array(train_data).reshape(-1, 600, 950, 1) # 600, 950은 image size
train_label = np.array(train_label)
test_data = np.array(test_data).reshape(-1, 600, 950, 1)
test_label = np.array(test_label)

In [ ]:
### 주의사항! : GPU 내 실행을 권장합니다. ###

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters = 16, strides = (1, 1), kernel_size = (3, 3),\
                         input_shape = (600, 950, 1), activation = 'relu', padding = 'valid'),
  tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
  tf.keras.layers.Conv2D(filters = 16, strides = (1, 1), kernel_size = (3, 3),\
                         activation = 'relu', padding = 'valid'),
  tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
  tf.keras.layers.Conv2D(filters = 16, strides = (1, 1), kernel_size = (3, 3),\
                         activation = 'relu', padding = 'valid'),
  tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
  tf.keras.layers.Conv2D(filters = 16, strides = (1, 1), kernel_size = (3, 3),\
                         activation = 'relu', padding = 'valid'),
  tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
  tf.keras.layers.Conv2D(filters = 16, strides = (1, 1), kernel_size = (3, 3),\
                         activation = 'relu', padding = 'valid'),
  tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(3 * 1024, activation='relu'),
  tf.keras.layers.Dense(96, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax'),
])

model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping() # 과적합 방지 적용
model.fit(train_data, train_label, epochs=10, validation_data=(test_data, test_label), callbacks=[early_stopping])
model.evaluate(test_data,  test_label, verbose=2)

model.save('CNN_Supervised_model.h5')

#### 4) Play Game

2-1 코드와 크게 다른점은 없고, 다만 make_data 함수를 새로 정의한 함수를 사용하고 모델을 다른 것을 사용한다는 차이점이 있다. move 함수는 2-1의 6)에서 정의했던 함수를 그대로 사용하면 된다.

In [ ]:
model = tf.keras.models.load_model('CNN_Supervised_model.h5')

In [ ]:
while True:
    while Is_GameOver(get_pic_array()): # 게임 오버 시 자동 재시작
        keyboard.press('space')
        if keyboard.is_pressed('n'): # n 입력하면 종료
            break
    keyboard.release('space')
    
    if keyboard.is_pressed('n'):
        break
    
    img = make_data()
    predict = np.argmax(model.predict(np.array([img]))[0]) # Tensor로 input을 넣어주기 위해 list를 numpy array로 변환
    move(predict)

## 3. Reinforcement Learning Process and Result

### 3-1. Measuring Method

#### 2) Make Data

input data를 정하는 방식이나 사용하게될 함수는 2-1의 1), 2), 3)과 동일하므로 여기서는 다루지 않는다. 참고로 여기서는 데이터를 수집할 필요가 없다. 다만, 2-1의 4)에서 사용했던 Is_GameOver함수는 그대로 사용한다. make_data함수는 image array를 함수 내부에서 만드는 것이 아니라 인자로 받는다는 점과 list가 아닌 numpy array를 반환한다는 점만 차이가 있다. 2-1의 5)에서 사용한 코드는 여기서 사용하지는 않고, 2-1의 6)에서는 move함수를 사용한다.

In [ ]:
def make_data(pic_array):
    try:
        if not Is_GameOver(pic_array):
            dist = measure_distance(pic_array)
            return np.array(dist)
        else:
            return np.array([-1]*12)
    except:
        return np.array([-1]*12)

#### 3) Define DQN Model

먼저 이 코드를 작성하는데 도움을 주신 리뷰어 분과, 이 코드의 뼈대를 공개해주신 팡요랩 컨텐츠 분들께 감사의 말씀을 드립니다. 

해당 코드는 다음 GitHub에 공개된 코드를 참조하였습니다. : https://github.com/seungeunrho/minimalRL/blob/master/dqn.py

해당 코드에 대한 자세한 설명은 다음 영상 내에 있습니다. : https://youtu.be/__NYgfkUr-M

이번 코드는 위의 스켈레톤 코드를 기반으로 이 프로젝트 내에 적용이 가능하도록 개조한 코드임을 먼저 밝힌다. 이번 파트는 하이퍼 파라미터 정의, ReplayBuffer class 작성 및 Q network class 작성을 하는 부분으로 이루어져있다. 여기에서는 Pytorch 프레임워크를 사용했었다.

In [ ]:
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### 하이퍼 파라미터 정의 ###

learning_rate = 0.01
gamma         = 0.99 # DQN에서 시간에 따른 감가율
buffer_limit  = 10000 # 전체 게임 데이터 셋 중 최근 몇 개의 dataset 중에서 랜덤 추출할지를 정하는 변수
batch_size    = 32 # 한 epoch 내 몇 개의 data를 랜덤 추출하여 학습에 사용할지를 정하는 변수

In [ ]:
### ReplayBuffer class 작성 ###

class ReplayBuffer():
    def __init__(self): # deque 자료형으로 선언
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition): # 데이터 buffer 내에 삽입하는 함수
        self.buffer.append(transition)
    
    def sample(self, n): # state, action, reward, next state, game over 여부 를 각각 저장 후 샘플링
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self): # buffer 내에 현재 존재하는 데이터의 개수 반환
        return len(self.buffer)

In [ ]:
### Q Network class 작성 ###

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(12, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon): # epsilon의 확률로 초기 랜덤 action
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,2)
        else : 
            return out.argmax().item()

In [ ]:
### class 객체 생성 및 시간 기록 세팅 ###

q = Qnet()
q_target = Qnet() # Q network와 동일한 target Q network도 같이 선언
q_target.load_state_dict(q.state_dict())
memory = ReplayBuffer()
time_series = [] # 각 epoch 별 기록을 저장하기 위한 리스트
best_time = -1 # 최고 기록

optimizer = optim.Adam(q.parameters(), lr=learning_rate)

#### 4) Training

ReplayBuffer 내의 데이터를 batch_size 만큼 임의 추출하여 Q_Network를 학습시키는 함수이다.

In [ ]:
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a) # 해당 action을 취했을 때의 q 값
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1) # 다음 state에서의 최대 q
        target = r + gamma * max_q_prime * done_mask # 다음 q에 감가율을 반영(각 게임의 마지막 action 에서는 done_mask = 0)
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # q 값과 target 값의 차이로 gradient descent 진행

#### 5) Play Game and Doing Reinforment Learning

게임을 반복실행하며 데이터를 모아 Q_Network를 학습시키고 기록을 계속해서 반환하는 코드이다. 중간 종료를 원하면 n을 입력하면 되며, 6단계 게임을 켜놓고 코드를 실행시키면 자동반복실행된다. 실행 전 앞에서 import했던 모든 라이브러리를 import하였는지 확인해보자.

In [ ]:
import time
import random

for n_epi in range(10000):
    epsilon = max(0.001, 0.99 ** n_epi) # random action 확률, 1 epoch마다 1%씩 감소
    while Is_GameOver(get_pic_array()):
        keyboard.press('space')
        if keyboard.is_pressed('n'):
            break
    keyboard.release('space')
            
    if keyboard.is_pressed('n'): # n 누르면 종료
        break
        
    pic_array = get_pic_array()
    s = make_data(pic_array)
    score = 0.0
    start = time.time()
    while True:
        a = q.sample_action(torch.from_numpy(s).float(), epsilon) # state에 대해서 예측 결과 반환, epsilon 확률로 random
        move(a)
        s_prime = make_data(pic_array)
        pic_array = get_pic_array()
        done_mask = 0.0 if Is_GameOver(pic_array) else 1.0
        r = 0.1 + (1 - done_mask) * -100.0 # survived action 0.1점, died action -100점 부여
        memory.put((s,a,r, s_prime, done_mask))
        s = s_prime

        score += r
        
        if Is_GameOver(pic_array): # 1 게임이 1 epoch에 해당
            break

    record = time.time() - start - 1 # 학습에 소요되는 시간은 빼고 생각

    time_series.append(record)
    
    if record > best_time: # 최고기록 업데이트
        best_time = record

    if memory.size()> 1000: # buffer 내에 1000개 이상 데이터 수집 시에만 학습 진행(초기 과적합 방지)
        train(q, q_target, memory, optimizer)

    if (n_epi + 1) % 10 == 0: # 10 epoch 마다 target Q network 동기화
        q_target.load_state_dict(q.state_dict())
        torch.save(q_target.state_dict(), './torch_model.pth') # 모델 저장
    print("n_episode :{}, score : {:.1f}, n_buffer : {}, best time : {:.2f} seconds".format(
                                                        n_epi+1, score, memory.size(), best_time)) # epoch 마다 상태 출력

### 3-2. CNN Method

#### 1) Collect Data

2-2에서 사용했던 방법과 거의 동일하나, pytorch를 사용할 예정이므로, shape가 약간 다른 데이터 생성 함수를 다시 작성하였다.

In [ ]:
def make_data(pic_array):
    cv2.rectangle(pic_array, (0,0),(949,50),(0,0,0), -1)
    pic_array = cv2.cvtColor(pic_array, cv2.COLOR_RGB2GRAY)
    pic_array = cv2.threshold(pic_array, 235, 255, cv2.THRESH_BINARY)
    return pic_array[1].reshape(1, 1, 600, 950)

#### 2) Define DQN Model

2-2에서 사용한 모델과 거의 비슷한 모델은 pytorch를 이용해 다시 구현해보았다.

In [ ]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        self.fc1 = nn.Linear(8352 , 96) # 600 * 950 image size 기준이므로, 다르다면 error 메시지를 보고 8352숫자를 조정하면 된다.
        self.fc2 = nn.Linear(96 , 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x.float())))
        x = self.pool(F.relu(self.conv2(x.float())))
        x = self.pool(F.relu(self.conv2(x.float())))
        x = self.pool(F.relu(self.conv2(x.float())))
        x = self.pool(F.relu(self.conv2(x.float())))
        x = x.view(-1, 8352)
        x = self.fc2(self.fc1(x))
        return x

    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,2)
        else : 
            return out.argmax().item()

#### 3) Training and Play Game

이후 과정은 3-1에서 사용한 코드와 거의 동일하다.

In [ ]:
learning_rate = 0.01
gamma         = 0.99 # DQN에서 시간에 따른 감가율
buffer_limit  = 10000 # 전체 게임 데이터 셋 중 최근 몇 개의 dataset 중에서 랜덤 추출할지를 정하는 변수
batch_size    = 32 # 한 epoch 내 몇 개의 data를 랜덤 추출하여 학습에 사용할지를 정하는 변수

class ReplayBuffer():
    def __init__(self): # deque 자료형으로 선언
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition): # 데이터 buffer 내에 삽입하는 함수
        self.buffer.append(transition)
    
    def sample(self, n): # state, action, reward, next state, game over 여부 를 각각 저장 후 샘플링
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s.reshape(1, 600 ,950))
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime.reshape(1, 600 ,950))
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self): # buffer 내에 현재 존재하는 데이터의 개수 반환
        return len(self.buffer)
        
q = Qnet()
q_target = Qnet() # Q network와 동일한 target Q network도 같이 선언
q_target.load_state_dict(q.state_dict())
memory = ReplayBuffer()
time_series = [] # 각 epoch 별 기록을 저장하기 위한 리스트
best_time = -1 # 최고 기록

optimizer = optim.Adam(q.parameters(), lr=learning_rate)

def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a) # 해당 action을 취했을 때의 q 값
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1) # 다음 state에서의 최대 q
        target = r + gamma * max_q_prime * done_mask # 다음 q에 감가율을 반영(각 게임의 마지막 action 에서는 done_mask = 0)
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # q 값과 target 값의 차이로 gradient descent 진행

In [ ]:
### 주의사항! GPU 실행을 권장합니다. ###

for n_epi in range(10000):
    epsilon = max(0.001, 0.99 ** n_epi) # random action 확률, 1 epoch마다 1%씩 감소
    while Is_GameOver(get_pic_array()):
        keyboard.press('space')
        if keyboard.is_pressed('n'):
            break
    keyboard.release('space')
            
    if keyboard.is_pressed('n'): # n 누르면 종료
        break
        
    pic_array = get_pic_array()
    s = make_data(pic_array)
    score = 0.0
    start = time.time()
    while True:
        a = q.sample_action(torch.from_numpy(s).float(), epsilon) # state에 대해서 예측 결과 반환, epsilon 확률로 random
        move(a)
        s_prime = make_data(pic_array)
        pic_array = get_pic_array()
        done_mask = 0.0 if Is_GameOver(pic_array) else 1.0
        r = 0.1 + (1 - done_mask) * -100.0 # survived action 0.1점, died action -100점 부여
        memory.put((s,a,r, s_prime, done_mask))
        s = s_prime

        score += r
        
        if Is_GameOver(pic_array): # 1 게임이 1 epoch에 해당
            break

    record = time.time() - start - 1.2 # 학습에 소요되는 시간은 빼고 생각

    time_series.append(record)
    
    if record > best_time: # 최고기록 업데이트
        best_time = record

    if memory.size()> 1000: # buffer 내에 1000개 이상 데이터 수집 시에만 학습 진행(초기 과적합 방지)
        train(q, q_target, memory, optimizer)

    if (n_epi + 1) % 10 == 0: # 10 epoch 마다 target Q network 동기화
        q_target.load_state_dict(q.state_dict())
        torch.save(q_target.state_dict(), './torch_model.pth') # 모델 저장
    print("n_episode :{}, score : {:.1f}, n_buffer : {}, best time : {:.2f} seconds".format(
                                                        n_epi+1, score, memory.size(), best_time)) # epoch 마다 상태 출력

문의 : hjy5405@naver.com